In [1]:
import numpy as np
import math as m
nm = 1.0e-9

### THE BELOW ASSUMES MATERIAL IN THE SAME PLANE AS PORE TOPS AND BOTS ARE THE SAME PHYSICAL SURFACE

#######BOOMER
text+="""L1[] = Translate{0,0,2e-8}{ Duplicata{Line{1};}};
L2[] = Translate{0,0,2e-8}{ Duplicata{Line{2};}};
L3[] = Translate{0,0,2e-8}{ Duplicata{Line{3};}};
L4[] = Translate{0,0,2e-8}{ Duplicata{Line{4};}};
Line Loop(2) = {L1[0],L2[0],L3[0],L4[0]};
Plane Surface(3) = {2};

L5[] = Translate{0,0,-1e-8}{ Duplicata{Line{1};}};
L6[] = Translate{0,0,-1e-8}{ Duplicata{Line{2};}};
L7[] = Translate{0,0,-1e-8}{ Duplicata{Line{3};}};
L8[] = Translate{0,0,-1e-8}{ Duplicata{Line{4};}};
Line Loop(3) = {L5[0],L6[0],L7[0],L8[0]};
Plane Surface(4) = {3};
"""

In [2]:
def Cylinder(centers,radius=1e-9,boxSize=15e-9):
    
    text = "lc = 1e-9;\n"
    lineNames = ""
    text+="radius = %s;\n"%(radius)
    text+="boxSize = %s;\n"%(boxSize)
    text+="Point(1) = { -boxSize, -boxSize, 0, lc};\n"
    text+="Point(2) = { -boxSize, boxSize, 0, lc};\n"
    text+="Point(3) = { boxSize, boxSize, 0, lc};\n"
    text+="Point(4) = { boxSize, -boxSize, 0, lc};\n"
    text+="Line(1) = {1,2};\n"
    text+="Line(2) = {2,3};\n"
    text+="Line(3) = {3,4};\n"
    text+="Line(4) = {4,1};\n"
    text+="Line Loop(1) = {1,2,3,4};\n"
    
    #text+="Plane Surface(1) = {1};\n"
    
    ptID=5 
    cirNum = 10
    for i,center in enumerate(centers):

        text+="Point(%s) = { %s, %s, 0, lc};\n"%(ptID,center[0],center[1])
        ptID+=1
        
        text+="Point(%s) = { %s, %s, 0, lc};\n"%(ptID,center[0]+radius,center[1])
        ptID+=1
        
        text+="Point(%s) = { %s, %s, 0, lc};\n"%(ptID,center[0],center[1]-radius)
        ptID+=1
        
        text+="Point(%s) = { %s, %s, 0, lc};\n"%(ptID,center[0]-radius,center[1])
        ptID+=1
        
        text+="Point(%s) = { %s, %s, 0, lc};\n"%(ptID,center[0],center[1]+radius)
        ptID+=1
                                
        text+="Circle(%s) = { %s, %s, %s};\n"%(cirNum,ptID-4,ptID-5,ptID-3)
        cirNum+=1
        
        text+="Circle(%s) = { %s, %s, %s};\n"%(cirNum,ptID-3,ptID-5,ptID-2)
        cirNum+=1
        
        text+="Circle(%s) = { %s, %s, %s};\n"%(cirNum,ptID-2,ptID-5,ptID-1)
        cirNum+=1
        
        text+="Circle(%s) = { %s, %s, %s};\n"%(cirNum,ptID-1,ptID-5,ptID-4)
        
        
        
        
        text+="Line Loop(%s) = {%s,%s,%s,%s};\n"%(i+5,cirNum-3,cirNum-2,cirNum-1,cirNum)
        text+="Plane Surface(%s) = {%s};\n\n"%(i+5,i+5)
        cirNum+=1
        
        if i==0:
            lineNames+= str(i+5)    
        else:
            lineNames+= "," + str(i+5)
    text+= "Plane Surface(1) = {1,%s};\n"%(lineNames)
    text+="NewSurf1[]=Translate{0,0,1e-8}{ Duplicata{Surface{1};}};"
    text+= "Physical Surface(1) = {1,NewSurf1[0]};\n"  ### This is the z=0 wall not pores
    text+= "Physical Surface(2) = {%s};\n"%(lineNames) ### These are the pore bots
    #####  BOOMER GOES HERE
    text+="""L1[] = Translate{0,0,2e-8}{ Duplicata{Line{1};}};
L2[] = Translate{0,0,2e-8}{ Duplicata{Line{2};}};
L3[] = Translate{0,0,2e-8}{ Duplicata{Line{3};}};
L4[] = Translate{0,0,2e-8}{ Duplicata{Line{4};}};
Line Loop(2) = {L1[0],L2[0],L3[0],L4[0]};
Plane Surface(3) = {2};

L5[] = Translate{0,0,-1e-8}{ Duplicata{Line{1};}};
L6[] = Translate{0,0,-1e-8}{ Duplicata{Line{2};}};
L7[] = Translate{0,0,-1e-8}{ Duplicata{Line{3};}};
L8[] = Translate{0,0,-1e-8}{ Duplicata{Line{4};}};
Line Loop(3) = {L5[0],L6[0],L7[0],L8[0]};
Plane Surface(4) = {3};
"""
    longString=""
    shorterString=""
    for i,center in enumerate(centers):
        text+="""out%s[]=Extrude{0,0,1e-8}{Surface{%s};};\n"""%(i+1,i+5)
        #text+="Volume(%s) = {out%s[1]};\n"%(i+5,i+1)
        shorterString +="out%s[0],"%(i+1)
        longString +="out%s[2],out%s[3],out%s[4],out%s[5],"%(i+1,i+1,i+1,i+1)
    longString=longString[:-1]
    shorterString=shorterString[:-1]
    #texts= text[:-2] #this is stupid ugly
    text+="Physical Surface(3) = {%s};\n"%(longString)  ### These are the pore walls
    text+="Physical Surface(4) = {%s};\n"%(shorterString)   ### These are the pore tops
    #text+="Physical Surface(5) = {3};\n"
    text+="Physical Surface(5) = {3};\n"  ### This is the top of the top reservoir
    text+="Physical Surface(6) = {4};\n"  ### This is the bottom of the bottom res
    text+="\n\n\n\n\n\n\n\n\n\n\n"
    return text


In [3]:
def Build(num, radius,bR,boxSize):
    centers = getCenters(num,radius,bR)
    text = Cylinder(centers,radius,boxSize)
    print "post cylinder"
    fileName = "tester.geo"
    fileLines = []
    fileLines.append(text)
    with open(fileName, "w") as text_file:
                for fileLine in fileLines:
                        text_file.write(fileLine)


    import os
    mshName = fileName.replace(".geo",".msh")
    xmlName = fileName.replace(".geo",".xml")
    print "pre OS"
    line = "gmsh -3 %s"%fileName
    os.system(line)
    line="dolfin-convert %s %s"%(mshName,xmlName)
    os.system(line)
    print "post OS"
    print xmlName
    return xmlName


In [4]:
def getCenters(num,radius,bR):
    numS = num**2
    i=0
    centers = [];

    if num%2!=0:
        while i <num:
            j=0
            x = radius*(-(3*num-4)/2+3*i)
            while j < num:
                
                y = radius*(-(3*num-4)/2+3*j)
                if (x**2)+(y**2)<bR**2:
                    this = False
                else:
                    centers.append((x,y))
                j+=1
            i+=1

    if num%2==0:
        while i <num:
            j=0
            x = radius*(-(3*num-4)/2+3*i-0.5)
            while j < num:
                y = radius*(-(3*num-4)/2+3*j-0.5)
                if (x**2)+(y**2)<bR**2:
                    this = False
                else:
                    centers.append((x,y))
                j+=1
            i+=1
    return centers



In [5]:
radius = 1e-9  #radius of the nanopores
bigR = 9e-9    #radius of the occlusion
boxSize = 15e-9 # size of the region being built
revH = 

In [6]:
Build(10,radius,bigR,boxSize)
print "nope"

post cylinder
pre OS
post OS
tester.xml
nope


In [7]:
print "sos"

sos
